# ACL debugging with Batfish

Network engineers are responsible for verifying that the ACLs in their networks are permitting and denying traffic as intended. This generally requires loading each ACL onto a lab device in order to test its behavior on a wide range of packets. Batfish makes it easy to see what each ACL will do with any packet, right down to the line of the ACL that matches it. We also provide a sanity check to ensure that every line in every ACL can match some packet that won't be matched by a previous line.

In this notebook, we will start by verifying that `host1` in our example network is correctly configured as a DNS server, then go on to check that all ACL lines are reachable across the network. The example network is shown below.

![example-network](https://raw.githubusercontent.com/batfish/batfish/master/test_rigs/example/example-network.png)

In [ ]:
# Initialize a network and snapshot
%run startup.py

NETWORK_NAME = "acl_debugging_network"
SNAPSHOT_NAME = "example_snapshot"
SNAPSHOT_PATH = "../test_rigs/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

## Debugging how ACLs treat a given packet

In this notebook we're operating from the perspective of `AS1` and would like to confirm that our ACLs are correctly configured to enable a DNS service hosted on `host1`. Packets from `AS1` destined for `host1` will hit two ACLs:
- Entering `AS2`, they will hit an ACL on the border router called `OUTSIDE_TO_INSIDE`
- Entering `host1`, they will hit an input ACL `filter::INPUT`

The Trace Filters question (`bfq.tracefilters`) is helpful for checking what ACLs do with particular packets. We can use it to test that ACLs `OUTSIDE_TO_INSIDE` and `filter::INPUT` permit DNS packets destined for `host1`.

### Parameters of Trace Filters

The Trace Filters question takes in the properties of a flow and a set of ACLs to trace. The answer shows what each ACL will do to that flow and why.

To specify the flow, you must provide the source IP address `srcIp` and the destination `dst` (hostname or IP address). Optionally, you can also specify IP protocols, ports, TCP flags, ICMP codes, and other properties ([see documentation for details](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.tracefilters)). The question will fill in any unspecified flow properties with default values.

The set of ACLs to examine can be narrowed down using these optional parameters:
- `nodeRegex` specifies the ACLs present on the set of nodes matching the given regex
- `filterRegex` specifies ACLs with names matching the given regex

If `nodeRegex` and `filterRegex` are left blank, Trace Filters will give results for every ACL in the network.

#### Tip: Pull up a table of hostnames to IP addresses to help fill in `srcIp` and `dst`
The Node Properties question achieves this nicely:

In [2]:
bfq.nodeProperties(nodeRegex=".*", propertySpec="canonical-ip").answer().frame()

,node,canonical-ip
0,as1border1,1.0.1.1
1,as1border2,1.0.2.1
2,as1core1,1.0.1.2
3,as2border1,2.1.1.1
4,as2border2,2.1.1.2
5,as2core1,2.1.2.1
6,as2core2,2.1.2.2
7,as2dept1,2.1.1.2
8,as2dist1,2.1.3.1
9,as2dist2,2.1.3.2


### Verifying that ACLs will not block DNS requests to `host1`

Suppose `as1core1` sends a DNS request destined for DNS server `host1`. Let's run Trace Filters to see how ACLs `OUTSIDE_TO_INSIDE` and `filter::INPUT` handle such a packet.

The result shows that both ACLs permit the packet, as expected, and also shows which line in each ACL accepts the packet.

In [3]:
as1core1_ip = "1.0.1.2"
host1_ip = "2.128.0.101"
nodes = "as2border1|host1"
acls = "OUTSIDE_TO_INSIDE|filter::INPUT"
bfq.tracefilters(dst=host1_ip, srcIp=as1core1_ip, dstProtocol="dns", nodeRegex=nodes, filterRegex=acls).answer().frame()

,node,filterName,flow,action,lineNumber,lineContent,trace
0,as2border1,OUTSIDE_TO_INSIDE,as2border1->[:1.0.1.2->0:2.128.0.101 proto: 53 dscp:UDP ecn:0 fragOff:0 length:0 state:0 flags: NEW,ACCEPT,2,permit ip any any,"Flow permitted by 'extended ipv4 access-list' named 'OUTSIDE_TO_INSIDE', index 2: permit ip any any"
1,host1,filter::INPUT,host1->[:1.0.1.2->0:2.128.0.101 proto: 53 dscp:UDP ecn:0 fragOff:0 length:0 state:0 flags: NEW,ACCEPT,0,-p udp --dport 53 -j ACCEPT,"Flow permitted by ACL named 'filter::INPUT', index 0: -p udp --dport 53 -j ACCEPT"


### Checking that an HTTP packet will be dropped
Meanwhile, other traffic from `AS1` should not reach `host1`. Let's run Trace Filters with an HTTP packet to check.

The result shows that although the packet is allowed into `AS2`, the input filter of `host1` drops it.

In [6]:
bfq.tracefilters(dst=host1_ip, srcIp=as1core1_ip, dstProtocol="tcp", dstPort=80, nodeRegex=nodes, filterRegex=acls).answer().frame()

,node,filterName,flow,action,lineNumber,lineContent,trace
0,as2border1,OUTSIDE_TO_INSIDE,as2border1->[:1.0.1.2->0:2.128.0.101 proto: 80 dscp:TCP ecn:0 fragOff:0 length:0 state:0 flags: NEW,ACCEPT,2,permit ip any any,"Flow permitted by 'extended ipv4 access-list' named 'OUTSIDE_TO_INSIDE', index 2: permit ip any any"
1,host1,filter::INPUT,host1->[:1.0.1.2->0:2.128.0.101 proto: 80 dscp:TCP ecn:0 fragOff:0 length:0 state:0 flags: NEW,REJECT,2,default,"Flow denied by ACL named 'filter::INPUT', index 2: default"


### Examining ACL behavior for other traffic from `AS1` to `host1`

We can run several more Trace Filters checks programmatically to look for other accepted flows. The next cell examines what happens to some other packets from `as1core1` destined for `host1`.

The result shows that DNS packets are accepted (UDP to port 53), as well as SSH packets (TCP to port 22). We do want `AS1` devices to be able to reach `host1` via SSH, so that is expected. All other packets tested are denied by one of the two ACLs along the path to `host1`.

In [5]:
common_ports = {
    'tcp': [1, 20, 22, 25, 80, 156, 179, 389, 443, 444],
    'udp': [7, 13, 37, 42, 49, 53, 107, 123, 156, 161]
}

for protocol in common_ports:
    for port in common_ports[protocol]:

        # Run Trace Filters with the packet specified by protocol and port
        trace_filters_result = bfq.tracefilters(dst=host1_ip, srcIp=as1core1_ip, dstProtocol=protocol, dstPort=port, nodeRegex=nodes, filterRegex=acls).answer().frame()

        # Create a filtered version of the result that only contains the ACLs that denied the packet
        rejects_only = trace_filters_result[trace_filters_result['action'].apply(lambda action:'REJECT' == action)]

        # If the filtered result is empty, then no ACLs denied the packet. Report that host1 accepted it.
        if len(rejects_only) == 0:
            print("Accepted: " + protocol + " on port " + str(port))

Accepted: tcp on port 22
Accepted: udp on port 53


## Ensuring all ACL lines are reachable

When debugging or editing ACLs, it can be useful to confirm that every line is reachable -- that is, it matches some set of packets that don't match earlier lines. Often unreachable ACL lines are symptomatic of past edits to the ACL that did not achieve their intent.

The ACL Reachability question (`bfq.aclReachability`) identifies unreachable ACL lines. Given no parameters, it will check every ACL in the network, but the scope can be narrowed down using parameters `aclNameRegex` and `nodeRegex` (see [documentation](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.aclReachability)).

For now, let's take a look at all the ACLs in the network.

In [3]:
acl_reach_answer = bfq.aclReachability().answer().frame()
acl_reach_answer

,aclSources,lines,blockedLineNum,blockingLineNums,differentAction,message
0,[as2dept1: RESTRICT_HOST_TRAFFIC_IN],"[permit ip 2.128.0.0 0.0.255.255 any, deny ip any any, permit icmp any any]",2,[1],True,ACLs { as2dept1: RESTRICT_HOST_TRAFFIC_IN } contain an unreachable line:\n [index 2] permit icmp any any\nBlocking line(s):\n [index 1] deny ip any any
1,[as2dept1: RESTRICT_HOST_TRAFFIC_OUT],"[permit ip any 2.128.0.0 0.0.255.255, deny ip 1.128.0.0 0.0.255.255 2.128.0.0 0.0.255.255, deny ip any any]",1,[0],True,ACLs { as2dept1: RESTRICT_HOST_TRAFFIC_OUT } contain an unreachable line:\n [index 1] deny ip 1.128.0.0 0.0.255.255 2.128.0.0 0.0.255.255\nBlocking line(s):\n [index 0] permit ip any 2.128.0.0 0.0.255.255


### Examining ACL reachability results
The answer identifies 2 unreachable lines. Let's take a closer look at the first one, line 2 in ACL `RESTRICT_HOST_TRAFFIC_IN` on node `as2dept1`. The `lines` column contains all the lines of the ACL, so the blocked and blocking lines can be found programmatically using their line numbers. The `message` column provides a human-readable result summary.

In [4]:
# Pull out first result
first_result = acl_reach_answer.iloc[0]

# Find the blocked and blocking lines from the lines column
lines = first_result['lines']
blocking_nums = first_result['blockingLineNums']
blocked_num = first_result['blockedLineNum']
blocking_lines = [lines[int(n)] for n in blocking_nums]
blocked_line = lines[blocked_num]
print('Results based on looking up lines in lines column:')
print('Blocked line: ' + blocked_line)
print('Blocking line(s): ' + str(blocking_lines))
print()

# Show the human-readable message
print("Message column:")
print(first_result['message'])

Results based on looking up lines in lines column:
Blocked line: permit icmp any any
Blocking line(s): ['deny   ip any any']

Message column:
ACLs { as2dept1: RESTRICT_HOST_TRAFFIC_IN } contain an unreachable line:
  [index 2] permit icmp any any
Blocking line(s):
  [index 1] deny   ip any any


In this case, the line is unreachable because previous line `deny ip any any` matches and denies all packets that `permit icmp any any` would have permitted. ACL Reachability also identifies:
- inherently unmatchable lines
- unreachable lines blocked by multiple partially blocking lines
- lines that have an uncertain impact because they contain an undefined or circular reference

## Get involved with the Batfish community

Thanks for checking out our ACL debugging examples! To get involved and learn more, check out the community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [Github](https://github.com/batfish/batfish). We would love to talk with you about Batfish or your network!